# Simulating IPFS and IPNS Systems

This notebook provides a Python-based simulation of IPFS (InterPlanetary File System) and IPNS (InterPlanetary Naming System) to test various linking strategies for storing and retrieving IPAROs.

The notebook uses three classes to simulate these systems:
- **IPARO**: Represents the storage object on IPFS.
- **IPNS**: Keeps track of the latest capture for different websites.
- **IPFS**: Simulates the hashing, storage, and retrieval of IPARO objects.

The goal of the simulation is to test various linking strategies.

In [7]:
# Importing the necessary libraries
import hashlib
import random

## IPARO Object

**Properties:**
- `CID`: The CID (Content Identifier) generated by IPFS.
- `Data`: The data of the capture.
- `Linked Node CID(s)`: The CID(s) of the nodes linked to it.

**Functions:**
- `get_cid`: Returns the CID of the IPARO.
- `get_linked_cids`: Returns the CID(s) of the linked node(s).
- `get_content`: Returns the content of the IPARO.
- `__str__`: Returns a string representation of the IPARO object.

In [8]:
class IPARO:
    def __init__(self, cid: str, linked_cids: list, content: str, timestamp: str):
        """
        Initialize an IPARO object with its CID, linked CID(s), and content.

        Args:
            cid (str): The CID of the IPARO.
            linked_cids (list): List of CIDs of linked nodes.
            content (str): The content of the IPARO.
        """
        self.__cid = cid
        self.__linked_cids = linked_cids
        self.__content = content
        self.__timestamp = timestamp

    def get_cid(self) -> str:
        '''
        Returns the CID of the IPARO.

        Returns:
            str: The CID of the IPARO.
        '''
        return self.__cid

    def get_linked_cids(self) -> list:
        '''
        Returns the CID(s) of linked nodes.

        Returns:
            list: List of linked node CIDs.
        '''
        return self.__linked_cids

    def get_content(self) -> str:
        '''
        Returns the content of the IPARO.

        Returns:
            str: The content stored in the IPARO.
        '''
        return self.__content

    def get_timestamp(self) -> str:
        '''
        Returns the timestamp of the IPARO

        Returns:
            str: The timestamp in seconds since the epoch
        '''
        return self.__timestamp

    def __str__(self):
        '''
        Returns a string representation of the IPARO object.

        Returns:
            str: A string containing the CID, linked CID(s), and content of the IPARO.
        '''
        iparo = {
            "CID": self.__cid,
            "Content": self.__content,
            "Linked CID(s)": self.__linked_cids,
            "Timestamp": self.__timestamp,
        }
        return str(iparo)

## IPNS Object

**Description:**
- The IPNS class stores and maps the latest CID of a website.
- Tracks the number of operations (get and update) performed.

**Functions:**
- `update`: Updates the latest CID of a website.
- `get_cid`: Retrieves the CID of the latest capture for a website.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [9]:
class IPNS:
    def __init__(self):
        """
        Initialize the IPNS object with an empty hashmap for storing CIDs 
        and counters for tracking operations.
        """
        self.data = {}
        self.update_count = 0
        self.get_count = 0

    def update(self, url, cid):
        '''
        Updates the latest CID for a given URL.

        Args:
            url (str): The URL of the website.
            cid (str): The CID of the latest capture.
        '''
        self.update_count += 1
        self.data[url] = cid

    def get_cid(self, url) -> str:
        '''
        Retrieves the latest CID for a given URL.

        Args:
            url (str): The URL of the website.

        Returns:
            str: The CID of the latest capture for the given URL.
        '''
        self.get_count += 1
        return self.data[url]

    def get_counts(self) -> dict:
        '''
        Returns the number of update and get operations performed.

        Returns:
            dict: Dictionary with the counts of update and get operations.
        '''
        counts = {"get": self.get_count, "update": self.update_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.update_count = 0
        self.get_count = 0

## IPFS Object

**Description:**
- The IPFS class stores the nodes and simulates the hashing, storage, and retrieval operations.
- Tracks the number of operations (hash, store, retrieve).

**Functions:**
- `hash`: Hashes the content of a node to generate its CID.
- `store`: Stores a node with its CID.
- `retrieve`: Retrieves a node using its CID.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [10]:
class IPFS:
    def __init__(self):
        '''
        Initialize the IPFS object with an empty hashmap for storing nodes
        and counters for tracking operations.
        '''
        self.data = {}
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def hash(self, content: str) -> str:
        '''
        Hashes the content to generate a CID.

        Args:
            content (str): The content of the node.

        Returns:
            str: The generated CID.
        '''
        sha256_hash = hashlib.sha256(content.encode()).hexdigest()
        self.hash_count += 1
        return 'Qm' + sha256_hash[:34]

    def store(self, cid: str, node: IPARO):
        '''
        Stores a node with its CID.

        Args:
            cid (str): The CID of the node.
            node (IPARO): The IPARO object to store.
        '''
        self.store_count += 1
        self.data[cid] = node

    def retrieve(self, cid) -> IPARO:
        '''
        Retrieves a node using its CID.

        Args:
            cid (str): The CID of the node to retrieve.

        Returns:
            IPARO: The retrieved IPARO object.
        '''
        self.retrieve_count += 1
        return self.data[cid]

    def get_counts(self) -> dict:
        '''
        Returns the number of hash, store, and retrieve operations performed.

        Returns:
            dict: Dictionary with counts of hash, store, and retrieve operations.
        '''
        counts = {"hash": self.hash_count, "store": self.store_count,
                  "retrieve": self.retrieve_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def reset_data(self):
        self.data = {}

    def get_data(self) -> dict:
        """Returns the data stored by IPFS (for debugging)."""
        return self.data

## Initialization and Operation Tracking

Here, we initialize the IPFS and IPNS objects and define a helper function `get_op_counts()` to display the number of operations performed.


In [11]:
# Initializing the simulated IPFS and IPNS
ipfs = IPFS()
ipns = IPNS()


def get_op_counts():
    '''
    Displays the number of operations performed by IPNS and IPFS.
    '''
    print("Number of operations IPNS performed:")
    print(ipns.get_counts())
    print("Number of operations IPFS performed:")
    print(ipfs.get_counts())

## Testing Different Linking Strategies

### 1. Linking to Only the Previous Node

In this test, each node will link only to the previous node in the chain. This strategy will be used to simulate a simple sequential storage system.


#### Storing Nodes

In [ ]:
import time

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)


# Automate the creation of additional nodes
for i in range(1, NODE_NUM):
    content = f"Node {i}"
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    linked_cids = [ipns.get_cid(URL)]  # Link to the previous node
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 99, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 0}


#### Retrieving Nodes

The following section tests retrieval of nodes by simulating a random node search.


In [13]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
while node.get_content() != target_content:
    node = ipfs.retrieve(node.get_linked_cids()[0])

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 96
Found node: {'CID': 'Qm846d79f21193d4c1468dc2558907e6525c', 'Content': 'Node 96', 'Linked CID(s)': ['Qmea1dffb495397a46e18e0ff2a92e69558d'], 'Timestamp': 1731104358.4868603}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 4}


### 2. Linking to all previous nodes

In this test, each node will link to all the previous nodes in the chain.

#### Storing Nodes

In [14]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

From here, there are 2 ways of creating a new node for this linking strategy, since this is a simulation, no data corruption can happen but that might not be true in practice. When retrieving the latest node which should contain the CIDs of all the previous node, two scenarios can happen:
1. The data is intact and the CIDs in the list is "correct" (which we really can't know for sure) and we can just add it to the new node we're creating
2. The data is corrupt and one or more of the CIDs is wrong or unfinished, in which case we have to recheck every CID to rebuild a new list of linked CIDs (not to mention fixing all the corrupted nodes)

So, for the purpose of this simulation, we will perform a check for every CID in the linked CID list of an IPARO to simulate the worst case scenario every time

In [15]:
# To automate adding the rest of the nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = latest_node_linked_cids
    for link_cid in latest_node_linked_cids:
        ipfs.retrieve(link_cid)
        # Checking and repairing nodes goes here
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 4953}


In this worst case scenario, where we have to retrieve and verify every CIDs in the linked CIDs of an IPARO, the retrieve count goes to almost 5000 (if we're storing 100 nodes)\
Of course this trade off makes it really easy to navigate to all the nodes just from the latest nodes

#### Retrieving nodes

The following section tests retrieval of nodes by simulating a random node search.

In [16]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
for linked_cid in linked_cids:
    node = ipfs.retrieve(linked_cid)
    if node.get_content() == target_content:
        break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 22
Found node: {'CID': 'Qm877343ea4ab00b3323700b051f9419986d', 'Content': 'Node 22', 'Linked CID(s)': ['Qm1176d4cec078d59cc8bb2a5b529f6aacf3', 'Qm268c92a3d269c40aa020d00333803995a2', 'Qm826ee083a8d73871489358396d22a3aa11', 'Qm36bd70dbe06441700a307d745120815fb4', 'Qm5be31b70ff47de31534525caee03106f0c', 'Qm1572c35b3548cb4e0230d8c18c2a766b8c', 'Qm9c711e2002fe38b3958c49e6ff62f78cc8', 'Qm9805d359f7f9c7f7c9d93ffc240078f79f', 'Qme83139924d55522aacfeea7b9865a816da', 'Qmaab3bb0bcf8db248d4ed51b03ee4ebd749', 'Qm65cb8dd36017980f5e26f894cc51f34a0c', 'Qm6651bc54b0a35124be2b57f85ff2254e93', 'Qm85d1cc6e11925cec263ec9391287dc4d5c', 'Qm809453b0554f6ac1de7b144c2f41223c6d', 'Qme28fe88d5b5fb51b17e639f46be8c7256f', 'Qm9f4f63540bde37f072c4ee501e317a2825', 'Qmd069e51ad8c18973ebdbfec7c8e938d4dd', 'Qmb1bfa7787dc83a9c3892af6ee375301e0f', 'Qm8ccee0f2b6931fcadd1884307a6deb2a0b', 'Qme9b888741268f253f8d7e2fe9c13bcad9e', 'Qmd562b4d9d6a8690cff0ee7896e9ebf0627', 'Qm26aa8f7f85d6095e8f

### Other strategies to be tested